In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import Ipynb_importer
import cnn
print(torch.__version__)

importing Jupyter notebook from cnn.ipynb
CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, d

In [2]:
batch_size = 128
learning_rate = 0.01
num_epoches = 20

In [3]:
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])

In [4]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=data_tf, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=data_tf)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
def train():
    model = cnn.CNN()

    if torch.cuda.is_available():
        print("CUDA is available")
        model = model.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9)
    
    for i in range(num_epoches):
        epoch = 0
        for img, label in train_loader:
            if torch.cuda.is_available():
                img = Variable(img).cuda()
                label = Variable(label).cuda()
            else:
                img = Variable(img)
                label = Variable(label)
            out = model(img)
            loss = criterion(out, label)
            print_loss = loss.data.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch += 1

        model.eval()
        eval_loss = 0
        eval_acc = 0
        for img, label in test_loader:
            img = Variable(img)
            if torch.cuda.is_available():
                img = Variable(img).cuda()
                label = Variable(label).cuda()
            else:
                img = Variable(img)
                label = Variable(label)
            out = model(img)
            loss = criterion(out, label)
            eval_loss += loss.data.item() * label.size(0)
            _, pred = torch.max(out, 1)
            num_correct = (pred == label).sum()
            eval_acc += num_correct.item()
        print('EPOCH: ', i + 1)
        print('Test Loss: {:.6f}, Acc: {:.6f}'.format(
            eval_loss / (len(test_dataset)),
            eval_acc / (len(test_dataset))
        ))
        i += 1

    torch.save(model, 'model.pth')

In [6]:
train()

CUDA is available
EPOCH:  1
Test Loss: 0.057134, Acc: 0.980100
EPOCH:  2
Test Loss: 0.032668, Acc: 0.990200
EPOCH:  3
Test Loss: 0.021693, Acc: 0.993200
EPOCH:  4
Test Loss: 0.027549, Acc: 0.991000
EPOCH:  5
Test Loss: 0.019438, Acc: 0.993800
EPOCH:  6
Test Loss: 0.035363, Acc: 0.989700
EPOCH:  7
Test Loss: 0.020524, Acc: 0.993800
EPOCH:  8
Test Loss: 0.027407, Acc: 0.992300
EPOCH:  9
Test Loss: 0.022215, Acc: 0.993800
EPOCH:  10
Test Loss: 0.023826, Acc: 0.994100
EPOCH:  11
Test Loss: 0.035618, Acc: 0.992200
EPOCH:  12
Test Loss: 0.032982, Acc: 0.990500
EPOCH:  13
Test Loss: 0.022878, Acc: 0.994300
EPOCH:  14
Test Loss: 0.024934, Acc: 0.993300
EPOCH:  15
Test Loss: 0.031029, Acc: 0.992600
EPOCH:  16
Test Loss: 0.027602, Acc: 0.993700
EPOCH:  17
Test Loss: 0.025094, Acc: 0.994600
EPOCH:  18
Test Loss: 0.025762, Acc: 0.994600
EPOCH:  19
Test Loss: 0.026771, Acc: 0.994600
EPOCH:  20
Test Loss: 0.027016, Acc: 0.994600


E:\anaconda\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
